# DACON 4 - 1차 제출
## COVID-19 Data EDA

### 말을 써야함 

- - - 
## Setting

In [1]:
import numpy as np 
import pandas as pd
import os
import glob
import re
import sys

import plotnine
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import folium
from folium import plugins
import branca
import json

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
import matplotlib.pylab as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

pio.renderers.default = "svg"

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

- - -

## Time related

### 1. Time

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=Time.date , y=Time.confirmed, name="comfirmed"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=Time.date, y=Time.deceased, name="deceased"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Confirmed & Deceased with Different Scale"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes
fig.update_layout(
    yaxis=dict(
        title="Confirmed",
    ),
    yaxis2=dict(
        title="Deceased",
    )
)

fig.show()

### 2. TimeProvince

In [ ]:
fig = px.bar(TimeProvince, x="province", y="confirmed", color="province",
    animation_frame="date", animation_group="province", range_y=[0,5500])
fig.show()

In [ ]:
fig_small = px.bar(TimeProvince, x="province", y="confirmed", color="province",
    animation_frame="date", animation_group="province", range_y=[0,300])
fig_small.show()

In [ ]:
fig_dec = px.bar(TimeProvince, x="province", y="deceased", color="province",
    animation_frame="date", animation_group="province", range_y=[0,100])
fig_dec.show()

In [ ]:
import matplotlib.pylab as plt

plt.rcParams["figure.figsize"] = (15,6)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['axes.grid'] = True

df = TimeProvince.iloc[TimeProvince[TimeProvince.date == "2020-02-18"].index[0]:]

for key, grp in df.groupby(['province']): 
    plt.plot(grp['date'], grp['confirmed']
            ,label = "in {}".format(key)
            )
plt.legend(loc='upper left')    
plt.xticks(rotation=45)

plt.show()

### 3. TimeAge

In [ ]:
fig_age = px.bar(TimeAge, x="age", y="confirmed", color="age",
    animation_frame="date", animation_group="age", range_y=[0,3000])
fig_age.show()

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['axes.grid'] = True

for key, grp in TimeAge.groupby(['age']): 
    plt.plot(grp['date'], grp['confirmed']
            ,label = "Age in {}".format(key)
            )
plt.legend(loc='upper right')    
plt.xticks(rotation=45)

plt.show()

### 4. TimeGender

In [ ]:
fig_sex = px.bar(TimeGender, x="sex", y="confirmed", color="sex",
    animation_frame="date", animation_group="sex", range_y=[0,6000])
fig_sex.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=TimeGender.date,
                y=TimeGender[TimeGender.sex == "male"].confirmed,
                name="Male",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=TimeGender.date,
                y=TimeGender[TimeGender.sex == "female"].confirmed,
                name="Female",
                line_color='deeppink',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(title_text="Gender; Confirmed")
fig.show()

### 5. SearchTrend


In [ ]:
SearchTrend.describe()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=SearchTrend.date,
                y=SearchTrend.cold,
                name="Cold",
                line_color='brown',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=SearchTrend.date,
                y=SearchTrend.flu,
                name="Flu",
                line_color='skyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=SearchTrend.date,
                y=SearchTrend.pneumonia,
                name="Pneumonia",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=SearchTrend.date,
                y=SearchTrend.coronavirus,
                name="Corona",
                line_color='red',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(title_text="Search Trend")
fig.show()

In [ ]:
fig = go.Figure()

df = SearchTrend.iloc[SearchTrend[SearchTrend.date == "2019-12-19"].index[0]:]

fig.add_trace(go.Scatter(
                x=df.date,
                y=df.cold,
                name="Cold",
                line_color='brown',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df.date,
                y=df.flu,
                name="Flu",
                line_color='skyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df.date,
                y=df.pneumonia,
                name="Pneumonia",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df.date,
                y=df.coronavirus,
                name="Corona",
                line_color='red',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(title_text="Search Trend Recent")
fig.show()

### 5-1. Search Trend + Confirmed

In [ ]:
grouped = (Time.groupby(['date'])
            .agg({'confirmed':'sum', 'deceased':'sum'})
            .reset_index())
merged = pd.merge(grouped,SearchTrend,
        how='left',
        on=('date')
        )

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged.date , y=merged.confirmed, name="Comfirmed"), secondary_y=False)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.cold, name="Cold trend"), secondary_y=True)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.flu, name="Flu trend"), secondary_y=True)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.pneumonia, name="Pneumonia trend"), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Confirmed & Search Trend")

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes
fig.update_layout(
    yaxis=dict(title="Confirmed"),
    yaxis2=dict(title="Trend")
)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged.date , y=merged.confirmed, name="Comfirmed"), secondary_y=False)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.coronavirus, name="Coronavirus trend"), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Confirmed & Corona Search Trend")

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes
fig.update_layout(
    yaxis=dict(title="Confirmed"),
    yaxis2=dict(title="Trend")
)

fig.show()

- - -

## Region related

### 1. Region

### 1-1. Count

In [ ]:
grouped = (Region[Region.province != "Korea"].groupby(['province'])
            .agg({'elementary_school_count':'sum', 'kindergarten_count':'sum', 'university_count':'sum', 'nursing_home_count':'sum'})
            .reset_index())
    
grouped.iloc[:,1:len(grouped)] = grouped.iloc[:,1:len(grouped)]/2 # Region province별 첫 행이 전체 sum값이 들어있음

import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = grouped.province

# Pie Chart !
fig = make_subplots(rows=2, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
                    ,subplot_titles=["kindergarten", "Elementary", "University", "Nursing Home"])
fig.add_trace(go.Pie(labels=labels, values=grouped.kindergarten_count, name="kindergarten count"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=grouped.elementary_school_count, name="Elementary School count"),
              1, 2)
fig.add_trace(go.Pie(labels=labels, values=grouped.university_count, name="University count"),
              2, 1)
fig.add_trace(go.Pie(labels=labels, values=grouped.nursing_home_count, name="Nursing Home count"),
              2, 2)

fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Region Information")
fig.show()

### 1-2. Ratio

In [ ]:
import plotly.express as px

fig = px.bar(Region, x="city", y="academy_ratio", color='province')
fig.update_layout(
    title='All Academy Ratio'
    , xaxis=dict(
        tickfont_size=14,
        tickangle = 45))
fig.show()

In [4]:
seoul = Region[Region.province=="Seoul"].reset_index().drop(0)
gyungi = Region[Region.province=="Gyeonggi-do"].reset_index().drop(0)
daegu = Region[Region.province=="Daegu"].reset_index().drop(0)

fig = go.Figure()
fig = make_subplots(rows=3, cols=1, subplot_titles=["Seoul", "Gyeonggi", "Daegu"])

fig.add_trace(go.Bar(x=seoul.city,
                y=seoul.academy_ratio,
                name = "seoul",
                marker_color='Blue'
                ),1, 1)
# fig.update_layout(xaxis=dict(tickfont_size=10, tickangle = 45))
        
fig.add_trace(go.Bar(x=gyungi.city,
                y=gyungi.academy_ratio,
                name = "gyungi",
                marker_color='Green'
                ),2, 1)
# fig.update_layout(xaxis=dict(tickfont_size=10, tickangle = 45))

fig.add_trace(go.Bar(x=daegu.city,
                y=daegu.academy_ratio,
                name = "daegu",
                marker_color='Red'
                ),3, 1)

fig.update_layout(
    title='Academy Ratio several provinces')
fig.show()

NameError: name 'Region' is not defined

In [ ]:

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=Region.city,
                y=Region.elderly_population_ratio,
                name='elderly population ratio'
                # marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=Region.city,
                y=Region.elderly_alone_ratio,
                name='elderly alone ratio',
                marker_color='lightsalmon'
                ))

fig.update_layout(
    title='Elderly Ratio All',
    xaxis=dict(tickfont_size=14, tickangle = 45),
    yaxis=dict(
        title='Ratio (%)',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0.75,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=seoul.city,
                y=seoul.elderly_population_ratio,
                name='elderly population ratio'
                # marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=seoul.city,
                y=seoul.elderly_alone_ratio,
                name='elderly alone ratio',
                marker_color='lightsalmon'
                ))

fig.update_layout(
    title='Elderly Ratio in Seoul',
    xaxis=dict(tickfont_size=14, tickangle = 45),
    yaxis=dict(
        title='Ratio (%)',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0.75,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=daegu.city,
                y=daegu.elderly_population_ratio,
                name='elderly population ratio'
                # marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=daegu.city,
                y=daegu.elderly_alone_ratio,
                name='elderly alone ratio',
                marker_color='lightsalmon'
                ))

fig.update_layout(
    title='Elderly Ratio in Daegu',
    xaxis=dict(tickfont_size=14, tickangle = 45),
    yaxis=dict(
        title='Ratio (%)',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0.75,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)
fig.show()

### 2. Weather

### 2-1. 온도 변화, 확진자수 증가 추세 (대구)

In [ ]:
grouped = (Time.groupby(['date'])
            .agg({'confirmed':'sum', 'deceased':'sum'})
            .reset_index())

daegu_weather = Weather[Weather.province=="Daegu"].reset_index()

merged = pd.merge(grouped,daegu_weather.iloc[:,3:],
        how='left',
        on=('date')
        )

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=merged.date , y=merged.confirmed, name="Comfirmed"), secondary_y=False)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.avg_temp, name="avg_temp"), secondary_y=True)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.min_temp, name="min_temp"), secondary_y=True)

fig.add_trace(
    go.Scatter(x=merged.date, y=merged.max_temp, name="max_temp"), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="Confirmed & Temp")

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes
fig.update_layout(
    yaxis=dict(title="Confirmed"),
    yaxis2=dict(title="temp")
)

fig.show()

### 2-2. 나머지 기온 변수, 확진자수 증가 추세 (대구)

In [ ]:
fig = px.scatter_matrix(merged[["confirmed","deceased","precipitation","max_wind_speed","most_wind_direction","avg_relative_humidity"]])
fig.update_layout(
    title='Weather Scatter Matrix')
fig.show()

- - -

## Case + Patient Related

### 1. Patient : 확진자 경로

#### 경로의 색깔 : 확진자가 접촉한 사람 수에 따라서

#### 접촉자 0 명 : 초록색

#### 접촉자 1~10 명 : 파란색

#### 접촉자 11~100명 : 노란색

#### 접촉자 100 초과 : 빨간색

#### 접촉자 수가 최다인 사람 : 검정색

In [ ]:
contnum = PatientInfo.iloc[:,[0,12]]
clr = []
for i in range(len(contnum)) :
    if contnum.contact_number[i] == 0 :
        clr.append("green")
    elif contnum.contact_number[i] <= 10 :
        clr.append("blue")
    elif contnum.contact_number[i] <= 100 :
        clr.append("yellow")
    elif contnum.contact_number[i] == max(contnum.contact_number) :
        clr.append("black")
    else : 
        clr.append("red")
contnum["color"] = clr
wcontnum = pd.merge(PatientRoute,contnum,
        how ='left',
        on = 'patient_id').fillna(0)
route_geo = gpd.GeoDataFrame(wcontnum,geometry=gpd.points_from_xy(PatientRoute.longitude,PatientRoute.latitude))

linerange = route_geo[route_geo['patient_id'] == route_geo['patient_id'].shift(1)].index
linerange = linerange.drop(linerange[-1])
lines = [ {
                "coordinates": [
                 [route_geo["longitude"][i] , route_geo["latitude"][i] ],
                 [route_geo["longitude"][i+1] , route_geo["latitude"][i+1],]
                   ],
                "dates": [
                 route_geo["date"][i],
                 route_geo["date"][i+1]
                   ],
                "color": route_geo["color"][i]
                     } for i in linerange ] 
features = [
     {
         'type': 'Feature',
         'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
                },
         'properties': {
            'times': line['dates'],
            'style': {
                'color' : line['color']
            },
            'icon' : 'circle',
            'iconstyle':{
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 3
                }
                }
        }
        for line in lines
    ]


m = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT6H', add_last_point=True).add_to(m)
m.save('with_color.html')
m

### 2. 시간에 따른 infection case 별 누적 확진자 수

In [ ]:
inf_count = pd.DataFrame(PatientInfo.infection_case.value_counts())
fig_0 = px.bar(inf_count,x=inf_count.index, y='infection_case')
fig_0.show()

In [ ]:
inf_case = PatientInfo.infection_case.unique()
for i in range(len(inf_case)) :
    globals()['case'+ str(i)] = PatientInfo[PatientInfo.infection_case == inf_case[i]]
    globals()['case'+str(i)] = pd.DataFrame(globals()['case'+ str(i)].groupby('confirmed_date').count().cumsum().patient_id).reset_index()
    globals()['case'+str(i)].columns = ['confirmed_date','case'+str(i)]   

merged = case0
for i in [x for x in range(len(inf_case)) if x != 0] :
    merged = pd.merge(merged , globals()['case'+str(i)],how= 'outer',on='confirmed_date')
merged = merged.sort_values(by = 'confirmed_date').reset_index()
merged = merged.drop('index',axis=1)
merged.columns = np.insert(inf_case,0,'confirmed_date')
merged = merged.fillna(method='pad')
merged = merged.fillna(0)
merged = merged.set_index('confirmed_date')
fig = merged.iplot(asFigure=True, kind='scatter')
fig.show()

#### overseas inflow 의 끝부분이 심상치않다 :)

# ^^ 감사합니다. I hope COVID-19 situation will get better soon. ^^*~~

# by 쇠똥구리